# VacationPy
### Setup & Configuration

In [1]:
# Dependencies and Setup
from urllib.parse import urlencode, urlparse, parse_qsl
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries
import os
from pathlib import Path

# Google developer API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame
* Making a new directory for output files

In [3]:
# Set Path and Read CSV file to DataFrame
import_file = "cities.csv"
import_file_path = '../WeatherPy/output_data/'
pathname = os.path.join(import_file_path, import_file)
weather_data = pd.read_csv(pathname)

# Creating the New Directory
output_dir = Path('output_data')
output_dir.mkdir(exist_ok=True)

In [4]:
weather_data

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Country_Name,UTC_DateTime,Hemisphere
0,EAST LONDON,-33.02,27.91,18.94,37,0,4.59,ZA,South Africa,2020-07-18 20:52:24,Southern
1,TIKSI,71.69,128.87,8.69,98,100,3.77,RU,Russian Federation,2020-07-18 20:57:01,Northern
2,CAPE CANAVERAL,28.41,-80.60,31.00,70,20,4.10,US,United States,2020-07-18 20:57:02,Northern
3,JAQUE,7.52,-78.17,28.07,77,98,1.25,PA,Panama,2020-07-18 20:57:02,Northern
4,MAR DEL PLATA,-38.00,-57.56,13.00,87,100,2.10,AR,Argentina,2020-07-18 20:53:58,Southern
...,...,...,...,...,...,...,...,...,...,...,...
574,LENINGRADSKIY,69.38,178.42,3.77,79,96,4.84,RU,Russian Federation,2020-07-18 20:58:13,Northern
575,TABOU,4.42,-7.35,24.50,86,5,3.16,CI,Côte d'Ivoire,2020-07-18 20:58:13,Northern
576,ANTIGUO MORELOS,22.55,-99.08,36.67,34,31,5.36,MX,Mexico,2020-07-18 20:58:13,Northern
577,ILHEUS,-14.79,-39.05,21.93,75,0,4.55,BR,Brazil,2020-07-18 20:58:13,Southern


### Set Access to GoogleMap
* Creating GoogleMapClient class object for requests

In [5]:
class GoogleMapClient(object):
    lat = None
    lng = None
    data_type ='json'
    location_query = None
    api_key=None
    
    def __init__(self, api_key=None, address_or_postal_code = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if g_key == None:
            raise Exception('Api key is required')
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        geocode_url = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {'address':self.location_query, 'key': self.api_key}
        params_url = urlencode(params)
        url = f"{geocode_url}?{params_url}"
        response = requests.get(url)
        if response.status_code not in range(200, 299):
            return {}
        latlng = {}
        try:
            latlng = response.json()['results'][0]['geometry']['location']
        except:
            pass
        lat, lng = latlng.get('lat'), latlng.get('lng')
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    def search(self, keyword='Mexican food', radius = 1000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng()
        end_point = f'https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}'
        params = {
                'key': self.api_key,
                'location': f"{lat},{lng}",
                'radius': radius,
                'keyword': keyword
                        }
        params_encoded = urlencode(params)
        places_endpoint = f"{end_point}?{params_encoded}"
        response = requests.get(places_endpoint)
        if response.status_code not in range(200, 299):
            return {}
        return response.json()
    
    def detail(self, place_id='ChIJv1DlfKQsDogRhm8x60TVJyo', fields=["name", "rating", "formatted_phone_number", "formatted_address"]):
        detail_end_point = f'https://maps.googleapis.com/maps/api/place/details/{self.data_type}'
        detail_params = {
                'place_id': f'{place_id}',
                'fields': ','.join(fields),
                'key': self.api_key
                }
        detail_params_encoded = urlencode(detail_params)
        detail_places_endpoint = f"{detail_end_point}?{detail_params_encoded}"
        response = requests.get(detail_places_endpoint)
        if response.status_code not in range(200, 299):
            return {}
        return response.json()      

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}

In [7]:
# Creating DataSet for all cities and adding HeatMap Layer
fig = gmaps.figure(layout=figure_layout)
locations = weather_data[['Lat', 'Lng']]
weight = weather_data['Humidity']
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=50, point_radius=2))
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='900px'))

![world_map](../images/world_map.png)

In [8]:
# Creating DataSet for US cities and adding HeatMap Layer
weather_us_cities = weather_data[weather_data['Country_Name']== 'United States']
locations = weather_us_cities[['Lat', 'Lng']]
weight = weather_us_cities['Humidity']
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=50, point_radius=2))
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='900px'))

![us_cities](../images/us_cities.png)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows will null values.

In [9]:
comfort_weather = weather_data[(weather_data['Humidity']<80) & (weather_data['Humidity']>70)
                              & (weather_data['Wind_Speed']<10) & (weather_data['Cloudiness']==0)].reset_index(drop=True)
comfort_weather

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Country_Name,UTC_DateTime,Hemisphere
0,ARRAIAL DO CABO,-22.97,-42.03,22.00,78,0,8.70,BR,Brazil,2020-07-18 20:57:04,Southern
1,PEMBA,-12.97,40.52,22.83,75,0,9.09,MZ,Mozambique,2020-07-18 20:57:05,Southern
2,HAJNOWKA,52.74,23.58,17.78,71,0,2.37,PL,Poland,2020-07-18 20:56:04,Northern
3,CARNARVON,-24.87,113.63,11.00,76,0,4.60,AU,Australia,2020-07-18 20:57:11,Southern
4,KOLPASHEVO,58.31,82.91,13.81,74,0,2.09,RU,Russian Federation,2020-07-18 20:57:23,Northern
5,COCKBURN TOWN,21.46,-71.14,28.31,78,0,7.27,TC,Turks and Caicos Islands,2020-07-18 20:57:28,Northern
6,NAMIBE,-15.20,12.15,19.72,78,0,2.37,AO,Angola,2020-07-18 20:57:31,Southern
7,CROTONE,39.09,17.12,22.40,75,0,0.87,IT,Italy,2020-07-18 20:57:48,Northern
8,BELOHA,-25.17,45.05,16.05,77,0,4.01,MG,Madagascar,2020-07-18 20:57:53,Southern
9,KAMARIOTISSA,40.48,25.47,22.00,78,0,8.81,GR,Greece,2020-07-18 20:58:08,Northern


### Hotel Map

In [10]:
# Adding Hotel Information for Cities fitting weather criteria
hotel_data = comfort_weather.filter(['City', 'Country_Name'])
for row in hotel_data.itertuples():
    client = GoogleMapClient(api_key = g_key, address_or_postal_code = f'{row.City}, {row.Country_Name}')
    try: 
        hotel_data.loc[row.Index, 'Lat'] = client.lat
        hotel_data.loc[row.Index, 'Lng'] = client.lng
        hotel_id = client.search('Hotel', radius=5000)['results'][0]['place_id']
        hotel_data.loc[row.Index, 'Hotel_Name'] = client.detail(place_id = hotel_id)['result']['name']
        hotel_data.loc[row.Index, 'Hotel_Rating'] = client.detail(place_id = hotel_id)['result']['rating']
        hotel_data.loc[row.Index, 'Hotel_Address'] = client.detail(place_id = hotel_id)['result']['formatted_address']
    except:
        pass
    print('Beginning data retrieval for ', row.Index+1, 'in', len(hotel_data), ' cities')

Beginning data retrieval for  1 in 12  cities
Beginning data retrieval for  2 in 12  cities
Beginning data retrieval for  3 in 12  cities
Beginning data retrieval for  4 in 12  cities
Beginning data retrieval for  5 in 12  cities
Beginning data retrieval for  6 in 12  cities
Beginning data retrieval for  7 in 12  cities
Beginning data retrieval for  8 in 12  cities
Beginning data retrieval for  9 in 12  cities
Beginning data retrieval for  10 in 12  cities
Beginning data retrieval for  11 in 12  cities
Beginning data retrieval for  12 in 12  cities


In [11]:
# Creating Hotel Info Dictionary
data_hotel_maplist = []
for row in hotel_data.itertuples():
    if row.Hotel_Rating >= 3.5:
        data_hotel_maplist.append(
            {'name': row.Hotel_Name, 'location': (row.Lat, row.Lng), 'city': row.City, 'rating': row.Hotel_Rating,'address': row.Hotel_Address})
hotel_locations = [hotel['location'] for hotel in data_hotel_maplist]

In [12]:
hotel_data.set_index('City', inplace=True)
hotel_data

,Country_Name,Lat,Lng,Hotel_Name,Hotel_Rating,Hotel_Address
City,,,,,,
ARRAIAL DO CABO,Brazil,-22.967337,-42.026810,Orlanova Hotel,4.1,"Av. Beira Mar, 111 - Prainha, Arraial do Cabo ..."
PEMBA,Mozambique,-12.973203,40.517801,Avani Pemba Beach Hotel,4.1,"Avenida da Marginal, Cabo Delgado Pemba, Pemba..."
HAJNOWKA,Poland,52.745140,23.581650,"Pension ""Forest Manor""",4.8,"Armii Krajowej 54, 17-200 Hajnówka, Poland"
CARNARVON,Australia,-24.887747,113.662183,Wintersun Caravan and Tourist Park,4.5,"546 Robinson St, Carnarvon WA 6701, Australia"
KOLPASHEVO,Russian Federation,58.313382,82.903638,"Гостиница ""Астория"" г. Колпашево",4.2,"Kommunisticheskaya Ulitsa, 15/1, Kolpashevo, T..."
COCKBURN TOWN,Turks and Caicos Islands,21.467458,-71.138910,Osprey Beach Hotel,4.4,"1 Duke St, Cockburn Town TKCA 1ZZ, Turks and C..."
NAMIBE,Angola,-15.197832,12.157554,iu Hotel Namibe,3.7,"Av. Eduardo Mondlane, Namibe, Angola"
CROTONE,Italy,39.080793,17.127110,Hotel San Giorgio,4.3,"Strada Statale 106 Jonica, 88900 Crotone KR, I..."
BELOHA,Madagascar,-25.169449,45.060656,NaN,NaN,NaN


In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Hotel Address</dt><dd>{address}</dd>
<dt>Hotel Ranking</dt><dd>{rating}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in data_hotel_maplist]
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

![hotel](../images/hotel_cities.png)